In [ ]:
import time

def inc(x):
    time.sleep(1)
    return x+ 1

In [ ]:
%%time

results = []
for x in range(10):
    result = inc(x)
    results.append(result)
    
print(results)

In [ ]:
%%time

# Using concurrent futures
from concurrent.futures import ThreadPoolExecutor
# from concurrent.futures import ProcessPoolExecutor

futures = []
with ThreadPoolExecutor(10) as e:
    for x in range(10):
        f = e.submit(inc, x)
        futures.append(f)
    # Wait
    results = [f.result() for f in futures]
print(len(results))

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
## Using Daskclients
import numpy as np

# client = Client(n_workers=10, threads_per_worker=1)
future = client.submit(np.arange, 100)

In [ ]:
future

In [ ]:
total = client.submit(np.sum, future)

In [ ]:
total

In [ ]:
total.result()

In [ ]:
del total

In [ ]:
del future

In [ ]:
import gc
gc.collect()

In [2]:
import time
import numpy as np

def load(x):
    time.sleep(0.1)
    v = np.random.randint(0,100,1)[0]
    return np.arange(1000000) + x + v

def process(x):
    time.sleep(0.1)
    v =  np.random.randint(0,100,1)[0]
    return x + 1 + v

def save(x):
    time.sleep(0.4)
    return None


In [ ]:
%%time

inputs = range(10)

for i in inputs:
    x = load(i)
    y = process(x)
    save(y)

In [ ]:
%%time

futures = []
for i in inputs:
    x = client.submit(load,i)
    y = client.submit(process, x)
    z = client.submit(save,y)
    futures.append(z)
    
result = [future.result() for future in futures]

In [ ]:
%%time

L = [client.submit(load,i) for i in range(1000)]
L2 = client.map(process, L)
L3 = client.map(save, L2)
L3
result = [future.result() for future in L3]

In [ ]:
del L

In [ ]:
del L2

In [ ]:
del L3

In [ ]:
del futures

In [ ]:
str(client)

In [ ]:
%%time

with Client(processes=False, threads_per_worker=12) as c:  # start local workers as threads
    L = [c.submit(load, i) for i in range(100)]
    L2 = c.map(process, L)
    L3 = c.map(save, L2)
    result = [future.result() for future in L3]

In [5]:
%%time

import dask
from dask import delayed



with dask.config.set(scheduler = 'processes'):
    L = [delayed(load)(i) for i in range(100)]
    L2 = [delayed(process)(l) for l in L]
    L3 = [delayed(save)(s) for s in L2]
    result = delayed(sum)([0 if l3 is None else 1 for l3 in L3])
    print(result.compute())

100
CPU times: user 19.3 ms, sys: 9.43 ms, total: 28.7 ms
Wall time: 228 ms


In [34]:
def simple_load(i):
    time.sleep(1)
    x = 0
    return(i)

In [37]:
%%time

vals = []
for i in range(10):
    l = delayed(simple_load)(i+1)
    vals.append(l)
x = sum(vals)
x.compute(scheduler = 'threads', num_workers=10)

CPU times: user 4.81 ms, sys: 2.22 ms, total: 7.03 ms
Wall time: 1.01 s


55

[Delayed('simple_load-85b9d814-6582-4edc-8254-452954c71847'),
 Delayed('simple_load-b06ce2ae-b270-4925-bd31-5b2c301cf741'),
 Delayed('simple_load-684f4e82-ba72-42f2-8dc1-f689808c8d27'),
 Delayed('simple_load-00657802-0ae9-480a-83a2-129e891cae2e'),
 Delayed('simple_load-36bf5380-d68f-401d-b206-df655893c4b2'),
 Delayed('simple_load-d94e6bef-0a1d-4c5a-80be-b80b30e0f8d1'),
 Delayed('simple_load-ea79a6fd-982e-4f9e-a4e5-32ff28864047'),
 Delayed('simple_load-88f911c3-250f-427e-b06c-b205a7e94c03'),
 Delayed('simple_load-861f93c3-f166-42fd-b0ea-dffc6eea58e2'),
 Delayed('simple_load-89fa1a7e-40c3-46a0-8f07-79e7e48e3bd6')]

In [41]:
from dask.distributed import Client
client = Client(n_workers=10, threads_per_worker=1)

In [42]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56725,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:56748,Total threads: 1
Dashboard: http://127.0.0.1:56752/status,Memory: 3.20 GiB
Nanny: tcp://127.0.0.1:56728,


In [44]:
client.scheduler.address

'tcp://127.0.0.1:56725'